## Imports

In [78]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [79]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Sep. 15th Sep. 22nd


In [80]:
filename = glob.glob('PBC*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [81]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [82]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [83]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [84]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,September-19-2024,Condo/Co-op,283 Wellington K,West Palm Beach,FL,33417.0,182000.0,2.0,2.0,Century Village West Palm Beach,1062.0,NaN,1972.0,NaN,171.0,525.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/283-...,Beaches MLS,RX-11018634,N,Y,26.710232,-80.133184


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [85]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [10]:
# # Filter dates between. If only using data looking back one week, this shouldn't change the df
# df = df[(df['SOLD DATE'] >= '2024-08-26') & (df['SOLD DATE'] <= '2024-09-02')]

In [86]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
19,PAST SALE,2024-09-23,Condo/Co-op,623 Sea Pine Way Unit C1,Greenacres,FL,33415.0,254000.0,2.0,2.0,Pine Ridge North II Condo Decl Filed 10-1-82,1005.0,365.0,1982.0,NaN,253.0,344.0,Sold,NaN,NaN,https://www.redfin.com/FL/Greenacres/623-Sea-P...,Beaches MLS,RX-11012027,N,Y,26.640170,-80.137817
23,PAST SALE,2024-09-23,Condo/Co-op,52 Yacht Club Dr #204,North Palm Beach,FL,33408.0,345000.0,2.0,2.0,Landings Condo LT,1130.0,NaN,1968.0,NaN,305.0,981.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/52-...,Beaches MLS,RX-11008673,N,Y,26.827637,-80.056070
65,PAST SALE,2024-09-23,Condo/Co-op,80 Venetian Dr Unit 201s,Delray Beach,FL,33483.0,532000.0,2.0,2.0,Miramar Garden Apt Condo,1169.0,NaN,1964.0,NaN,455.0,725.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/80-Vene...,Beaches MLS,RX-10942380,N,Y,26.460320,-80.062104
63,PAST SALE,2024-09-23,Condo/Co-op,3800 Washington Rd #311,West Palm Beach,FL,33405.0,288287.0,1.0,1.5,Portofino South Condo Estates OF South Palm Bea,1077.0,NaN,1971.0,NaN,268.0,1347.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/3800...,Beaches MLS,RX-10954557,N,Y,26.677294,-80.049335
43,PAST SALE,2024-09-23,Condo/Co-op,6795 Huntington Ln #205,Delray Beach,FL,33446.0,110000.0,2.0,2.0,Huntington Lakes Sec Two Condos,990.0,NaN,1985.0,NaN,111.0,964.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/6795-Hu...,Beaches MLS,RX-10985107,N,Y,26.464302,-80.150119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,PAST SALE,2024-09-17,Condo/Co-op,4111 Oak Terrace Dr,Greenacres,FL,33463.0,198000.0,2.0,2.0,Oak Court Condo,883.0,NaN,1981.0,NaN,224.0,597.0,Sold,NaN,NaN,https://www.redfin.com/FL/Greenacres/4111-Oak-...,Beaches MLS,RX-11000164,N,Y,26.610922,-80.139373
39,PAST SALE,2024-09-17,Condo/Co-op,370 Golfview Rd #703,North Palm Beach,FL,33408.0,456950.0,2.0,2.0,Seamark Condo,1393.0,NaN,2003.0,NaN,328.0,947.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/370...,Beaches MLS,RX-10999831,N,Y,26.835537,-80.063121
53,PAST SALE,2024-09-17,Condo/Co-op,3111 Gardens East Dr #23,Palm Beach Gardens,FL,33410.0,200000.0,2.0,2.0,Royal Point Manor West Condo,1088.0,NaN,1977.0,NaN,184.0,470.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/3...,Beaches MLS,RX-10978920,N,Y,26.823790,-80.083155
51,PAST SALE,2024-09-17,Condo/Co-op,1700 Crestwood Ct S #1707,Royal Palm Beach,FL,33411.0,280000.0,2.0,2.0,Grand View AT Crestwood Condo,1200.0,NaN,2001.0,NaN,233.0,532.0,Sold,NaN,NaN,https://www.redfin.com/FL/Royal-Palm-Beach/170...,Beaches MLS,RX-10985683,N,Y,26.683178,-80.243430


In [87]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [88]:
df_top_ten = df.head(10)

## Collect Agent Information

In [89]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [90]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [91]:
response_df = pd.DataFrame(response_list)

In [92]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [93]:
df_top_ten = merged_df

## Current Week's Values

In [94]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

67


In [95]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$593,041


In [96]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$343


In [97]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$39,733,762


In [14]:
# # Define the date range
# start_date = '2024-05-20'
# end_date = '2024-05-26'

# # Filter rows where 'SOLD DATE' is within the specified range
# second_week_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

## Previous Week's Values

In [27]:
# Define the date range
start_date = '2024-09-08'
end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [35]:
filtered_df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [43]:
# comparison week total sales
print(f'Comparison week filtered sales: {(len(filtered_df))}')
print('-----------')
print(f'Comparison week average sales price: ${filtered_df["PRICE"].mean():,.02f}')
print('-----------')
print(f'Comparison week average psf: ${filtered_df["$/SQUARE FEET"].mean():,.02f}')
print('-----------')
print(f'Comparison week sales total: ${filtered_df["PRICE"].sum():,.02f}')
print('-----------')

Comparison week filtered sales: 91
-----------
Comparison week average sales price: $504,396.37
-----------
Comparison week average psf: $318.00
-----------
Comparison week sales total: $45,900,070.00
-----------


In [44]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
91
Input Previous Week Condo Average Sales Price:
504396
Input Previous Week Condo Average PSF:
318
Input Previous Week Condo Sales total (ex: 198_000_000)
45_900_000


In [98]:
print('Input county name:')
county = input()

Input county name:
Palm Beach County


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [99]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [100]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [101]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [102]:
pd.set_option('display.max_columns',None)

In [103]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-09-19,Condo/Co-op,400 S Ocean Blvd Unit R-12,Boca Raton,FL,33432.0,5100000.0,4.0,4.5,Excelsior,4295.0,NaN,2002.0,NaN,1187.0,5532.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/400-S-Oce...,Beaches MLS,RX-10967449,N,Y,26.34646,-80.070587,https://www.redfin.com/FL/Boca-Raton/400-S-Oce...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Michael Botta,One Sotheby's International Realty,None,None,1,orange


In [104]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [105]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-09-19,Condo/Co-op,400 S Ocean Blvd Unit R-12,Boca Raton,FL,33432.0,5100000.0,4.0,4.5,Excelsior,4295.0,NaN,2002.0,NaN,1187.0,5532.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/400-S-Oce...,Beaches MLS,RX-10967449,N,Y,26.346460,-80.070587,https://www.redfin.com/FL/Boca-Raton/400-S-Oce...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Michael Botta,One Sotheby's International Realty,None,None,1,orange
1,PAST SALE,2024-09-18,Condo/Co-op,3100 S Ocean Blvd Unit 502n,Palm Beach,FL,33480.0,4501600.0,2.0,2.5,Palm Beach Hampton Condo,2244.0,NaN,1985.0,NaN,2006.0,2535.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3100-S-Oc...,Beaches MLS,RX-10994867,N,Y,26.609772,-80.037138,https://www.redfin.com/FL/Palm-Beach/3100-S-Oc...,Scott Gordon,Douglas Elliman (Palm Beach),Mindy Gordon,Douglas Elliman (Palm Beach),Angela Guitard,Coldwell Banker,None,None,2,blue
2,PAST SALE,2024-09-18,Condo/Co-op,2770 S Ocean Blvd Unit 403 N,Palm Beach,FL,33480.0,4500000.0,3.0,4.5,Twenty Seven Seventy South Ocean Blvd,4370.0,NaN,1991.0,NaN,1030.0,6583.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Beaches MLS,RX-10986279,N,Y,26.622316,-80.037453,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Dana Koch,The Corcoran Group,None,None,Lawrence Moens,Lawrence A. Moens Associates,None,None,3,blue
3,PAST SALE,2024-09-17,Condo/Co-op,2784 S Ocean Blvd Unit 101s,Palm Beach,FL,33480.0,2857500.0,3.0,2.5,Cove Condo,2189.0,NaN,1975.0,NaN,1305.0,2607.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2784-S-Oc...,Beaches MLS,RX-10998894,N,Y,26.618753,-80.039845,https://www.redfin.com/FL/Palm-Beach/2784-S-Oc...,Samantha Curry,Douglas Elliman (Palm Beach),None,None,Elisabeth Sabol-Dono,Compass Florida LLC (PB),None,None,4,blue
4,PAST SALE,2024-09-18,Condo/Co-op,3720 S Ocean Blvd #703,Highland Beach,FL,33487.0,1550000.0,2.0,2.5,Toscana West Tower II,2292.0,NaN,2002.0,NaN,676.0,2639.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-10927623,N,Y,26.408254,-80.066146,https://www.redfin.com/FL/Highland-Beach/3720-...,Vincent Morea,The Keyes Company,None,None,Rodger Morelli,RPM LUXE REALTY,None,None,5,blue
5,PAST SALE,2024-09-17,Condo/Co-op,5310 N Ocean Dr #601,Singer Island,FL,33404.0,1475000.0,3.0,3.0,One Singer Island Condominium,2904.0,NaN,2006.0,NaN,508.0,5284.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5310-N...,Beaches MLS,RX-10992914,N,Y,26.806528,-80.034213,https://www.redfin.com/FL/Riviera-Beach/5310-N...,Dermot OBrien,"Sotheby's Intl. Realty, Inc.",None,None,Martha Jolicoeur,Douglas Elliman (Wellington),None,None,6,blue
6,PAST SALE,2024-09-17,Condo/Co-op,3300 S Ocean Blvd Unit 301s,Palm Beach,FL,33480.0,1350000.0,2.0,2.0,Meridian OF Palm Beach Condo,1902.0,NaN,1981.0,NaN,710.0,2045.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3300-S-Oc...,Beaches MLS,RX-11005007,N,Y,26.602038,-80.037703,https://www.redfin.com/FL/Palm-Beach/3300-S-Oc...,Pamela Gottfried,Douglas Elliman (Palm Beach),Joan Wenzel-Messing,Douglas Elliman (Palm Beach),Christopher Small,Platinum Properties/The Keyes,None,None,7,blue
7,PAST SALE,2024-09-20,Condo/Co-op,99 SE Mizner Blvd #719,Boca Raton,FL,33432.0,890000.0,3.0,2.0,Palmetto Place Condo,1500.0,139793.0,2003.0,NaN,593.0,1475.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/99-SE-Miz...,Beaches MLS,RX-10996556,N,Y,26.348949,-80.083947,https://www.redfin.com/FL/Boca-Raton/99-SE-Miz...,Anna Zukowski,Palmetto Realty LLC,None,None,Shakie Iskandarian,Beautiful Homes Realty Inc,No

In [106]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [107]:
m.save('index.html')

## Data snagger

In [108]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [109]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-09-19,Condo/Co-op,400 S Ocean Blvd Unit R-12,Boca Raton,FL,33432.0,5100000.0,4.0,4.5,Excelsior,4295.0,NaN,2002.0,NaN,1187.0,5532.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/400-S-Oce...,Beaches MLS,RX-10967449,N,Y,26.34646,-80.070587,https://www.redfin.com/FL/Boca-Raton/400-S-Oce...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Michael Botta,One Sotheby's International Realty,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [110]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PBC_condo_sales_week_ending_09232024


# CREATE TEMPLATE 

In [111]:
muni_set = set(df_top_ten['CITY'])

In [112]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [113]:
df_top_ten.reset_index(inplace=True,drop=True)

In [114]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [115]:
top_sale

'Excelsior at 400 S Ocean Blvd Unit R-12 in Boca Raton'

In [116]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [117]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [118]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-09-19,Condo/Co-op,400 S Ocean Blvd Unit R-12,Boca Raton,FL,33432.0,5100000.0,4.0,4.5,Excelsior,4295.0,NaN,2002.0,NaN,1187.0,5532.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/400-S-Oce...,Beaches MLS,RX-10967449,N,Y,26.346460,-80.070587,https://www.redfin.com/FL/Boca-Raton/400-S-Oce...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Michael Botta,One Sotheby's International Realty,None,None,1,orange,Excelsior at 400 S Ocean Blvd Unit R-12 in Boc...
1,PAST SALE,2024-09-18,Condo/Co-op,3100 S Ocean Blvd Unit 502n,Palm Beach,FL,33480.0,4501600.0,2.0,2.5,Palm Beach Hampton Condo,2244.0,NaN,1985.0,NaN,2006.0,2535.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3100-S-Oc...,Beaches MLS,RX-10994867,N,Y,26.609772,-80.037138,https://www.redfin.com/FL/Palm-Beach/3100-S-Oc...,Scott Gordon,Douglas Elliman (Palm Beach),Mindy Gordon,Douglas Elliman (Palm Beach),Angela Guitard,Coldwell Banker,None,None,2,blue,Palm Beach Hampton Condo at 3100 S Ocean Blvd ...
2,PAST SALE,2024-09-18,Condo/Co-op,2770 S Ocean Blvd Unit 403 N,Palm Beach,FL,33480.0,4500000.0,3.0,4.5,Twenty Seven Seventy South Ocean Blvd,4370.0,NaN,1991.0,NaN,1030.0,6583.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Beaches MLS,RX-10986279,N,Y,26.622316,-80.037453,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Dana Koch,The Corcoran Group,None,None,Lawrence Moens,Lawrence A. Moens Associates,None,None,3,blue,Twenty Seven Seventy South Ocean Blvd at 2770 ...
3,PAST SALE,2024-09-17,Condo/Co-op,2784 S Ocean Blvd Unit 101s,Palm Beach,FL,33480.0,2857500.0,3.0,2.5,Cove Condo,2189.0,NaN,1975.0,NaN,1305.0,2607.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2784-S-Oc...,Beaches MLS,RX-10998894,N,Y,26.618753,-80.039845,https://www.redfin.com/FL/Palm-Beach/2784-S-Oc...,Samantha Curry,Douglas Elliman (Palm Beach),None,None,Elisabeth Sabol-Dono,Compass Florida LLC (PB),None,None,4,blue,Cove Condo at 2784 S Ocean Blvd Unit 101s in P...
4,PAST SALE,2024-09-18,Condo/Co-op,3720 S Ocean Blvd #703,Highland Beach,FL,33487.0,1550000.0,2.0,2.5,Toscana West Tower II,2292.0,NaN,2002.0,NaN,676.0,2639.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-10927623,N,Y,26.408254,-80.066146,https://www.redfin.com/FL/Highland-Beach/3720-...,Vincent Morea,The Keyes Company,None,None,Rodger Morelli,RPM LUXE REALTY,None,None,5,blue,Toscana West Tower II at 3720 S Ocean Blvd #70...
5,PAST SALE,2024-09-17,Condo/Co-op,5310 N Ocean Dr #601,Singer Island,FL,33404.0,1475000.0,3.0,3.0,One Singer Island Condominium,2904.0,NaN,2006.0,NaN,508.0,5284.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5310-N...,Beaches MLS,RX-10992914,N,Y,26.806528,-80.034213,https://www.redfin.com/FL/Riviera-Beach/5310-N...,Dermot OBrien,"Sotheby's Intl. Realty, Inc.",None,None,Martha Jolicoeur,Douglas Elliman (Wellington),None,None,6,blue,One Singer Island Condominium at 5310 N Ocean ...
6,PAST SALE,2024-09-17,Condo/Co-op,3300 S Ocean Blvd Unit 301s,Palm Beach,FL,33480.0,1350000.0,2.0,2.0,Meridian OF Palm Beach Condo,1902.0,NaN,1981.0,NaN,710.0,2045.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3300-S-Oc...,Beaches MLS,RX-11005007,N,Y,26.602038,-80.037703,https://www.redfin.com/FL/Palm-Beach/3300-S-Oc...,Pamela Gottfried,Douglas Elliman (Palm Beach),Joan Wenzel-Messing,Douglas Elliman (Palm Beach),Christopher Small,Platinum Properties/The Keyes,None,None,7,blue,Meridian OF Palm Beach Condo at 3300 S Ocean B...
7,PAST SALE,2024-09-20,Condo/Co-op,99 SE Mizner Blvd

In [119]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Excelsior closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $725,000M to $5,100,000M
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, South Palm Beach, Palm Beach, Boca Raton, Singer Island, Highland Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 67 condo sales totaling $39,733,762 million from Sep. 15th to Sep. 22nd. The previous week, brokers closed 91 condo sales totaling $45,900,000.

Last week’s units sold for an average of $593,041, higher than the $504,396 average pric

In [120]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [121]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [122]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [123]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-09-19,Condo/Co-op,400 S Ocean Blvd Unit R-12,Boca Raton,FL,33432.0,5100000.0,4.0,4.5,Excelsior,4295.0,NaN,2002.0,NaN,1187.0,5532.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/400-S-Oce...,Beaches MLS,RX-10967449,N,Y,26.346460,-80.070587,https://www.redfin.com/FL/Boca-Raton/400-S-Oce...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Michael Botta,One Sotheby's International Realty,None,None,1,orange,Excelsior at 400 S Ocean Blvd Unit R-12 in Boc...
1,PAST SALE,2024-09-18,Condo/Co-op,3100 S Ocean Blvd Unit 502n,Palm Beach,FL,33480.0,4501600.0,2.0,2.5,Palm Beach Hampton Condo,2244.0,NaN,1985.0,NaN,2006.0,2535.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3100-S-Oc...,Beaches MLS,RX-10994867,N,Y,26.609772,-80.037138,https://www.redfin.com/FL/Palm-Beach/3100-S-Oc...,Scott Gordon,Douglas Elliman (Palm Beach),Mindy Gordon,Douglas Elliman (Palm Beach),Angela Guitard,Coldwell Banker,None,None,2,blue,Palm Beach Hampton Condo at 3100 S Ocean Blvd ...
2,PAST SALE,2024-09-18,Condo/Co-op,2770 S Ocean Blvd Unit 403 N,Palm Beach,FL,33480.0,4500000.0,3.0,4.5,Twenty Seven Seventy South Ocean Blvd,4370.0,NaN,1991.0,NaN,1030.0,6583.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Beaches MLS,RX-10986279,N,Y,26.622316,-80.037453,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Dana Koch,The Corcoran Group,None,None,Lawrence Moens,Lawrence A. Moens Associates,None,None,3,blue,Twenty Seven Seventy South Ocean Blvd at 2770 ...
3,PAST SALE,2024-09-17,Condo/Co-op,2784 S Ocean Blvd Unit 101s,Palm Beach,FL,33480.0,2857500.0,3.0,2.5,Cove Condo,2189.0,NaN,1975.0,NaN,1305.0,2607.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2784-S-Oc...,Beaches MLS,RX-10998894,N,Y,26.618753,-80.039845,https://www.redfin.com/FL/Palm-Beach/2784-S-Oc...,Samantha Curry,Douglas Elliman (Palm Beach),None,None,Elisabeth Sabol-Dono,Compass Florida LLC (PB),None,None,4,blue,Cove Condo at 2784 S Ocean Blvd Unit 101s in P...
4,PAST SALE,2024-09-18,Condo/Co-op,3720 S Ocean Blvd #703,Highland Beach,FL,33487.0,1550000.0,2.0,2.5,Toscana West Tower II,2292.0,NaN,2002.0,NaN,676.0,2639.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-10927623,N,Y,26.408254,-80.066146,https://www.redfin.com/FL/Highland-Beach/3720-...,Vincent Morea,The Keyes Company,None,None,Rodger Morelli,RPM LUXE REALTY,None,None,5,blue,Toscana West Tower II at 3720 S Ocean Blvd #70...
5,PAST SALE,2024-09-17,Condo/Co-op,5310 N Ocean Dr #601,Singer Island,FL,33404.0,1475000.0,3.0,3.0,One Singer Island Condominium,2904.0,NaN,2006.0,NaN,508.0,5284.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5310-N...,Beaches MLS,RX-10992914,N,Y,26.806528,-80.034213,https://www.redfin.com/FL/Riviera-Beach/5310-N...,Dermot OBrien,"Sotheby's Intl. Realty, Inc.",None,None,Martha Jolicoeur,Douglas Elliman (Wellington),None,None,6,blue,One Singer Island Condominium at 5310 N Ocean ...
6,PAST SALE,2024-09-17,Condo/Co-op,3300 S Ocean Blvd Unit 301s,Palm Beach,FL,33480.0,1350000.0,2.0,2.0,Meridian OF Palm Beach Condo,1902.0,NaN,1981.0,NaN,710.0,2045.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3300-S-Oc...,Beaches MLS,RX-11005007,N,Y,26.602038,-80.037703,https://www.redfin.com/FL/Palm-Beach/3300-S-Oc...,Pamela Gottfried,Douglas Elliman (Palm Beach),Joan Wenzel-Messing,Douglas Elliman (Palm Beach),Christopher Small,Platinum Properties/The Keyes,None,None,7,blue,Meridian OF Palm Beach Condo at 3300 S Ocean B...
7,PAST SALE,2024-09-20,Condo/Co-op,99 SE Mizner Blvd

In [124]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: Excelsior closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $725,000M to $5,100,000M
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, South Palm Beach, Palm Beach, Boca Raton, Singer Island, Highland Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 67 condo sales totaling $39,733,762 million from Sep. 15th to Sep. 22nd. The previous week, brokers closed 91 condo sales totaling $45,900,000.

Last week’s units sold for an average of $593,041, higher than the $504,396 average pric

In [133]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Boca-Raton/400-S-Ocean-Blvd-33432/unit-R-12/home/190095737


In [134]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Palm-Beach/3100-S-Ocean-Blvd-33480/unit-502N/home/42583355


In [127]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Palm-Beach/3546-S-Ocean-Blvd-33480/unit-405/home/42585227


In [128]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Highland-Beach/3420-S-Ocean-Blvd-33487/unit-9Z/home/42388715


In [129]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Palm-Beach/3100-S-Ocean-Blvd-33480/unit-502N/home/42583355


In [130]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Riviera-Beach/5310-N-Ocean-Dr-33404/unit-601/home/42653320


In [131]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-09-19,Condo/Co-op,400 S Ocean Blvd Unit R-12,Boca Raton,FL,33432.0,5100000.0,4.0,4.5,Excelsior,4295.0,NaN,2002.0,NaN,1187.0,5532.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/400-S-Oce...,Beaches MLS,RX-10967449,N,Y,26.346460,-80.070587,https://www.redfin.com/FL/Boca-Raton/400-S-Oce...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Michael Botta,One Sotheby's International Realty,None,None,1,orange,Excelsior at 400 S Ocean Blvd Unit R-12 in Boc...
1,PAST SALE,2024-09-18,Condo/Co-op,3100 S Ocean Blvd Unit 502n,Palm Beach,FL,33480.0,4501600.0,2.0,2.5,Palm Beach Hampton Condo,2244.0,NaN,1985.0,NaN,2006.0,2535.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3100-S-Oc...,Beaches MLS,RX-10994867,N,Y,26.609772,-80.037138,https://www.redfin.com/FL/Palm-Beach/3100-S-Oc...,Scott Gordon,Douglas Elliman (Palm Beach),Mindy Gordon,Douglas Elliman (Palm Beach),Angela Guitard,Coldwell Banker,None,None,2,blue,Palm Beach Hampton Condo at 3100 S Ocean Blvd ...
2,PAST SALE,2024-09-18,Condo/Co-op,2770 S Ocean Blvd Unit 403 N,Palm Beach,FL,33480.0,4500000.0,3.0,4.5,Twenty Seven Seventy South Ocean Blvd,4370.0,NaN,1991.0,NaN,1030.0,6583.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Beaches MLS,RX-10986279,N,Y,26.622316,-80.037453,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Dana Koch,The Corcoran Group,None,None,Lawrence Moens,Lawrence A. Moens Associates,None,None,3,blue,Twenty Seven Seventy South Ocean Blvd at 2770 ...
3,PAST SALE,2024-09-17,Condo/Co-op,2784 S Ocean Blvd Unit 101s,Palm Beach,FL,33480.0,2857500.0,3.0,2.5,Cove Condo,2189.0,NaN,1975.0,NaN,1305.0,2607.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2784-S-Oc...,Beaches MLS,RX-10998894,N,Y,26.618753,-80.039845,https://www.redfin.com/FL/Palm-Beach/2784-S-Oc...,Samantha Curry,Douglas Elliman (Palm Beach),None,None,Elisabeth Sabol-Dono,Compass Florida LLC (PB),None,None,4,blue,Cove Condo at 2784 S Ocean Blvd Unit 101s in P...
4,PAST SALE,2024-09-18,Condo/Co-op,3720 S Ocean Blvd #703,Highland Beach,FL,33487.0,1550000.0,2.0,2.5,Toscana West Tower II,2292.0,NaN,2002.0,NaN,676.0,2639.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-10927623,N,Y,26.408254,-80.066146,https://www.redfin.com/FL/Highland-Beach/3720-...,Vincent Morea,The Keyes Company,None,None,Rodger Morelli,RPM LUXE REALTY,None,None,5,blue,Toscana West Tower II at 3720 S Ocean Blvd #70...
5,PAST SALE,2024-09-17,Condo/Co-op,5310 N Ocean Dr #601,Singer Island,FL,33404.0,1475000.0,3.0,3.0,One Singer Island Condominium,2904.0,NaN,2006.0,NaN,508.0,5284.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5310-N...,Beaches MLS,RX-10992914,N,Y,26.806528,-80.034213,https://www.redfin.com/FL/Riviera-Beach/5310-N...,Dermot OBrien,"Sotheby's Intl. Realty, Inc.",None,None,Martha Jolicoeur,Douglas Elliman (Wellington),None,None,6,blue,One Singer Island Condominium at 5310 N Ocean ...
6,PAST SALE,2024-09-17,Condo/Co-op,3300 S Ocean Blvd Unit 301s,Palm Beach,FL,33480.0,1350000.0,2.0,2.0,Meridian OF Palm Beach Condo,1902.0,NaN,1981.0,NaN,710.0,2045.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3300-S-Oc...,Beaches MLS,RX-11005007,N,Y,26.602038,-80.037703,https://www.redfin.com/FL/Palm-Beach/3300-S-Oc...,Pamela Gottfried,Douglas Elliman (Palm Beach),Joan Wenzel-Messing,Douglas Elliman (Palm Beach),Christopher Small,Platinum Properties/The Keyes,None,None,7,blue,Meridian OF Palm Beach Condo at 3300 S Ocean B...
7,PAST SALE,2024-09-20,Condo/Co-op,99 SE Mizner Blvd

## Time on Market Calculator

In [135]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 6, 10) ## List (Earlier) date
date2 = datetime(2024, 9, 18) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

100


## Clean CSV for Datawrapper Chart

In [136]:
chart_df = df_top_ten

In [137]:
chart_df = chart_df.fillna(" ")

In [138]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [139]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [140]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [141]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [142]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [143]:
csv_date_string = today.strftime("%m_%d_%Y")

In [144]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")